<a href="https://colab.research.google.com/github/Raaj-4320/the-exam-paper-generator/blob/main/topic_based_question_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00


In [16]:
import requests
import pandas as pd
import re

In [19]:
topic = "Matrix"

In [28]:
def search_wikipedia(topic):
    search_url = f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={topic}&format=json"
    response = requests.get(search_url)
    search_results = response.json()
    return search_results['query']['search']

def clean_snippet(snippet):
    soup = BeautifulSoup(snippet, "html.parser")
    text = soup.get_text()
    return text

def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Convert to lowercase
    tokens = [word.lower() for word in tokens]

    # Remove stop words and non-alphabetic tokens
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

    return ' '.join(tokens)

# Example usage
topic = "Regression in machine learning"
search_results = search_wikipedia(topic)

# Extract titles from the search results
titles = [result['title'] for result in search_results]

# Print related topics (titles)
print("Related topics:")
for title in titles:
    print(title)

Related topics:
Regression analysis
Decision tree learning
Support vector machine
Feature (machine learning)
Machine learning
Outline of machine learning
Supervised learning
Statistical learning theory
Hyperparameter (machine learning)
Regularization (mathematics)


In [29]:
import requests

def get_wikipedia_extract(title, length="short"):
    # Define the endpoint
    endpoint = "https://en.wikipedia.org/w/api.php"

    # Set the extract length parameters
    if length == "short":
        exchars = 5000  # Limit to approximately one paragraph
        params = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "exchars": exchars,
            "explaintext": True
        }
    elif length == "medium":
        exsentences = 20  # Limit to approximately two paragraphs
        params = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "exsentences": exsentences,
            "explaintext": True
        }
    else:
        params = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "exintro": True,
            "explaintext": True
        }

    # Make the request to the Wikipedia API
    response = requests.get(endpoint, params=params)
    data = response.json()

    # Extract the page information
    pages = data['query']['pages']
    page = next(iter(pages.values()))  # Get the first (and only) page

    # Extract the extract text
    extract = page.get("extract", "No extract found.")

    return extract

def search_wikipedia(topic):
    search_url = f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={topic}&format=json"
    response = requests.get(search_url)
    search_results = response.json()
    return search_results['query']['search']

# Example usage
topic = "Regression in machine learning"
search_results = search_wikipedia(topic)

# Extract titles from the search results
titles = [result['title'] for result in search_results]

corpus = {}
# Iterate through each title and get a paragraph (full extract) for each
for title in titles:
    extract = get_wikipedia_extract(title, length="full")
    print(f"Title: {title}\nExtract:\n{extract}\n")
    corpus[title] = extract
    # the_text = (f"Title: {title}\nExtract:\n{extract}\n")
# print(corpus)

Title: Support vector machine
Extract:
In machine learning, support vector machines (SVMs, also support vector networks) are supervised max-margin models with associated learning algorithms that analyze data for classification and regression analysis. Developed at AT&T Bell Laboratories by Vladimir Vapnik with colleagues (Boser et al., 1992, Guyon et al., 1993, Cortes and Vapnik, 1995, Vapnik et al., 1997) SVMs are one of the most studied models, being based on statistical learning frameworks of VC theory proposed by Vapnik (1982, 1995) and Chervonenkis (1974). 
In addition to performing linear classification, SVMs can efficiently perform a non-linear classification using what is called the kernel trick, which represent the data only through a set of pairwise similarity comparisons between the original data observations and representing the data by these transformed coordinates in the higher dimensional feature space. Thus, SVMs use the kernel trick to implicitly map their inputs into 

# OpenAI

In [1]:
from openai import OpenAI

client = OpenAI(
    api_key=''
)

ModuleNotFoundError: No module named 'openai'

In [ ]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role':'system','content':'you are a good teacher who sets a good exam papers'},
        {'role': 'user','content':f'create 50 multiple choice questions of the given text {corpus} with easy complexity, and mention all the answer with the question itself marked as .'}
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Sure, here are 50 easy multiple choice questions based on the given text:\n\n1. What is machine learning (ML)?\na) A field of study in artificial intelligence\nb) A method for explicitly programming computers\nc) A type of hardware used in computer systems\nd) A tool for manipulating data directly\n\n2. What is the primary focus of machine learning?\na) Generating data from scratch\nb) Learning from data and generalizing\nc) Following explicit instructions only\nd) Avoiding statistical algorithms\n\n3. What type of networks have surpassed many previous approaches in performance?\na) Biological neural networks\nb) Artificial neural networks\nc) Quantum neural networks\nd) Statistical neural networks\n\n4. In what fields does machine learning find application?\na) Only in natural language processing\nb) Only in computer vision\nc) In various fields like medicine and agriculture\nd) Solely in email filtering\n\n5. What is not a source of machine learning met

# Manual Question generator

In [35]:
import re

def convert_to_question(sentence):
    # List of auxiliary verbs to identify questions
    question_words = ["is", "are", "am", "was", "were", "can", "could", "shall", "should", "will", "would", "do", "does", "did", "have", "has", "had", "may", "might", "must"]

    # Split the sentence into words
    words = sentence.split()

    # Regex to identify verbs
    verb_regex = re.compile(r"\bis\b|\bare\b|\bam\b|\bwas\b|\bwere\b|\bcan\b|\bcould\b|\bshall\b|\bshould\b|\bwill\b|\bwould\b|\bdo\b|\bdoes\b|\bdid\b|\bhave\b|\bhas\b|\bhad\b|\bmay\b|\bmight\b|\bmust\b")

    # Find the first auxiliary verb in the sentence
    for i, word in enumerate(words):
        if verb_regex.match(word.lower()):
            # Ensure we handle 'have' correctly
            if word.lower() == "have" and i != 0:
                question_sentence = "Do " + " ".join(words[:i] + [words[i]] + words[i+1:]) + "?"
            else:
                # Move the auxiliary verb to the front
                question_sentence = word + " " + " ".join(words[:i] + words[i+1:]) + "?"
            return question_sentence.capitalize()

    # Handle sentences without auxiliary verbs by adding 'do'
    if len(words) > 1:
        return "Do " + sentence + "?"
    else:
        return sentence + "?"

def paragraph_to_questions(paragraph):
    # Split the paragraph into sentences
    sentences = re.split(r'(?<=[.!?]) +', paragraph)
    questions = [convert_to_question(sentence.strip()) for sentence in sentences if sentence.strip()]
    return questions

# Example input paragraph
paragraph = """
I bought apples from market and bananas from super-market
"""

# Generate questions
questions = paragraph_to_questions(paragraph)
for question in questions:
    print(question)


Do I bought apples from market and bananas from super-market?


# --------------------------------------------------------------------------------

# New question generator